In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/usr0/home/naveenr/projects/spurious_concepts/ConceptBottleneck/')
sys.path.append('/usr0/home/naveenr/projects/spurious_concepts')

In [3]:
import torch
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
import torch.nn as nn
import torch.optim as optim
import pickle
import matplotlib.pyplot as plt
import torch.nn.functional as F
from PIL import Image
from captum.attr import visualization as viz
from matplotlib.colors import LinearSegmentedColormap
import cv2
from copy import copy 
import itertools
import json
import argparse 
import secrets
import subprocess
import shutil 
from torch.nn.utils import prune
import resource 

/usr0/home/naveenr/miniconda3/envs/concepts_spurious/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from src.images import *
from src.util import *
from src.models import *
from src.plot import *

In [5]:
torch.set_num_threads(1)

In [6]:
is_jupyter = 'ipykernel' in sys.modules
if is_jupyter:
    encoder_model='small7'
    seed = 42
    num_concept_combinations = 8

    num_objects = 4
else:
    parser = argparse.ArgumentParser(description="Synthetic Dataset Experiments")


    parser.add_argument('--encoder_model', type=str, default='small3', help='Encoder model')
    parser.add_argument('--seed', type=int, default=42, help='Random seed')
    parser.add_argument('--num_concept_combinations', type=int, default=1, help='Number of concept combinations')
    parser.add_argument('--num_objects', type=int, default=4, help='Number of objects/which synthetic dataset')

    args = parser.parse_args()
    encoder_model = args.encoder_model 
    seed = args.seed 
    num_concept_combinations = args.num_concept_combinations 
    num_objects = args.num_objects

dataset_name = "synthetic_object/synthetic_{}".format(num_objects)

parameters = {
    'seed': seed, 
    'encoder_model': encoder_model ,
    'debugging': False,
    'dataset_name': dataset_name,
    'num_concept_combinations': num_concept_combinations
}
print(parameters)
torch.cuda.set_per_process_memory_fraction(0.5)
resource.setrlimit(resource.RLIMIT_AS, (20 * 1024 * 1024 * 1024, -1))


{'seed': 42, 'encoder_model': 'small7', 'debugging': False, 'dataset_name': 'synthetic_object/synthetic_4', 'num_concept_combinations': 8}


In [7]:
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

In [8]:
train_loader, val_loader, test_loader, train_pkl, val_pkl, test_pkl = get_data(num_objects,encoder_model=encoder_model,dataset_name=dataset_name)

In [9]:
test_images, test_y, test_c = unroll_data(test_loader)

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
def generate_random_combinations(L, K):
    random.seed(seed)    
    # Generate all possible combinations
    all_combinations = list(itertools.product([0, 1], repeat=L))
    random.shuffle(all_combinations)

    return all_combinations[:K]

In [12]:
random_combinations = generate_random_combinations(num_objects,num_concept_combinations)
random_full_combinations = []
for c in random_combinations:
    random_full_combinations.append([])
    for d in c:
        random_full_combinations[-1].append(d)
        random_full_combinations[-1].append(1-d)
formatted_combinations = []
for r in random_full_combinations:
    formatted_combinations.append(str(int("".join([str(i) for i in r]),2)))

In [13]:
command_to_run = "python train_cbm.py -dataset {} -epochs {} -num_attributes {} --encoder_model {} -num_classes 2 -seed {} --concept_restriction {}".format(dataset_name,50,num_objects*2,encoder_model,seed," ".join(formatted_combinations))

In [14]:
subprocess.run("cd ../../ConceptBottleneck && {}".format(command_to_run),shell=True)

Files to delete are ['c284ddb2']
Namespace(attr_loss_weight=1.0, batch_size=32, bottleneck=False, ckpt='0', concept_restriction=[106, 150, 102, 105, 169, 153, 165, 149], connect_CY=False, data_dir='../../../datasets/synthetic_object/synthetic_4/preprocessed', dataset='cub', encoder_model='small7', end2end=True, epochs=50, exp='Joint', expand_dim=0, expand_dim_encoder=0, experiment_name='CUB', freeze=False, image_dir='images', load_model='none', log_dir='../models/synthetic_object/synthetic_4/0543854a/joint', lr=0.05, mask_loss_weight=1.0, n_attributes=8, n_class_attr=2, no_img=False, normalize_loss=True, num_classes=2, num_middle_encoder=0, one_batch=False, optimizer='sgd', pretrained=False, resampling=False, save_step=1000, scale_factor=1.5, scale_lr=5, scheduler='none', scheduler_step=30, seed=42, three_class=False, train_addition='', train_variation='none', uncertain_labels=False, use_attr=True, use_aux=True, use_relu=False, use_sigmoid=True, use_unknown=False, weight_decay=0.0004, 

CompletedProcess(args='cd ../../ConceptBottleneck && python train_cbm.py -dataset synthetic_object/synthetic_4 -epochs 50 -num_attributes 8 --encoder_model small7 -num_classes 2 -seed 42 --concept_restriction 106 150 102 105 169 153 165 149', returncode=0)

In [15]:
def get_most_recent_file(directory):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    if not files:
        return None

    most_recent_file = max(files, key=os.path.getmtime)
    
    return most_recent_file


In [16]:
most_recent_data = get_most_recent_file("../../models/model_data/")
rand_name = most_recent_data.split("/")[-1].replace(".json","")
results_file = "../../results/correlation/{}.json".format(rand_name)
delete_same_dict(parameters,"../../results/correlation")

In [17]:
joint_location = "../../models/synthetic_object/synthetic_{}/{}/joint/best_model_{}.pth".format(num_objects,rand_name,seed)
joint_model = torch.load(joint_location,map_location='cpu')

if 'encoder_model' in parameters and 'mlp' in parameters['encoder_model']:
    joint_model.encoder_model = True

r = joint_model.eval()

In [18]:
joint_model = joint_model.to(device)

In [19]:
torch.cuda.empty_cache()

## Compute Accuracy

In [20]:
train_acc =  get_accuracy(joint_model,run_joint_model,train_loader)
val_acc = get_accuracy(joint_model,run_joint_model,val_loader)
test_acc =get_accuracy(joint_model,run_joint_model,test_loader)

In [21]:
in_distro = 0
correct_in_distro = 0 

out_distro = 0
correct_out_distro = 0

device = 'cuda' if torch.cuda.is_available() else 'cpu'

with torch.no_grad():  # Use torch.no_grad() to disable gradient computation

    for data in test_loader:
        x, y, c = data
        y_pred, c_pred = run_joint_model(joint_model, x.to(device))
        c_pred = torch.stack([i.detach() for i in c_pred])
        c_pred = torch.nn.Sigmoid()(c_pred)

        c_pred = c_pred.numpy().T
        y_pred = logits_to_index(y_pred.detach())

        c = torch.stack([i.detach() for i in c]).numpy().T

        in_distribution = []

        for i in range(len(c)):
            binary_c = c[i]
            combo = str(int("".join([str(i) for i in binary_c]),2))

            if combo in formatted_combinations:
                in_distribution.append(True)
            else:
                in_distribution.append(False)
        
        in_distro += in_distribution.count(True) * len(c[0])
        out_distro += in_distribution.count(False) * len(c[0])

        in_distribution = np.array(in_distribution)

        correct_in_distro += np.sum(np.clip(np.round(c_pred[in_distribution]),0,1) == c[in_distribution]).item() 
        correct_out_distro += np.sum(np.clip(np.round(c_pred[~in_distribution]),0,1) == c[~in_distribution]).item() 



In [22]:
concept_accuracies = []

# Try and flip each concept
for concept_num in range(num_objects*2):
# Set this concept_num to 1 (which sets the corresponding thing to 0)
    total_flipped = 0
    total_points = 0

    with torch.no_grad():  # Use torch.no_grad() to disable gradient computation

        for data in test_loader:
            x, y, c = data
            y_pred, c_pred = run_joint_model(joint_model, x.to(device))
            c_pred = torch.stack([i.detach() for i in c_pred]).numpy().T
            y_pred = logits_to_index(y_pred.detach())

            c = torch.stack([i.detach() for i in c]).numpy().T

            in_distribution = []

            for i in range(len(c)):
                # Just look for errors where binary_c = 1 in prediction

                binary_c = c[i]

                if binary_c[concept_num] == 1:
                    in_distribution.append(True)
                else:
                    in_distribution.append(False)
            
            in_distribution = np.array(in_distribution)
            total_points += np.sum(in_distribution) 
            total_flipped += np.sum(np.clip(np.round(c_pred[in_distribution,concept_num]),0,1) == c[in_distribution,concept_num]) 
            
    concept_accuracies.append(total_flipped/total_points)

In [26]:
final_data = {
    'train_accuracy': train_acc, 
    'val_accuracy': val_acc, 
    'test_accuracy': test_acc, 
    'in_distro': correct_in_distro/in_distro, 
    'num_in_distro': in_distro, 
    'out_distro': correct_out_distro/out_distro, 
    'num_out_distro': out_distro, 
    'concept_accuracies': concept_accuracies,
    'combinations': formatted_combinations,
    'parameters': parameters,  
}

In [27]:
final_data

{'train_accuracy': 0.6923828125,
 'val_accuracy': 0.697265625,
 'test_accuracy': 0.685546875,
 'in_distro': 0.9839494163424124,
 'num_in_distro': 2056,
 'out_distro': 0.7289215686274509,
 'num_out_distro': 2040,
 'concept_accuracies': [0.9881422924901185,
  0.752895752895753,
  0.9964664310954063,
  0.4585152838427948,
  0.7666666666666667,
  0.9889705882352942,
  1.0,
  0.9814814814814815],
 'combinations': ['106', '150', '102', '105', '169', '153', '165', '149'],
 'parameters': {'seed': 42,
  'encoder_model': 'small7',
  'debugging': False,
  'dataset_name': 'synthetic_object/synthetic_4',
  'num_concept_combinations': 8}}

In [28]:
json.dump(final_data,open(results_file,"w"))